# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by `model.profile`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [2]:
import pandas as pd
from pymc3 import *
from pymc3.math import exp
from pymc3.distributions.timeseries import *

returns = pd.read_csv(get_data('SP500.csv'), index_col=0, parse_dates=True)

n = returns.shape[0]

with Model() as model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)
    nu = Exponential('nu', 1. / 10)
    s = GaussianRandomWalk('s', sigma ** -2, shape=n)
    r = StudentT('r', nu, lam=exp(-2 * s), observed=returns)
    


Then we call the `profile` function and summarize its return values.

In [3]:
model.profile(model.logpt).summary()


Function profiling
  Message: /Users/ckrapu/miniconda3/envs/pymc3-dev/lib/python3.7/site-packages/pymc3-3.5-py3.7.egg/pymc3/model.py:910
  Time in 1000 calls to Function.__call__: 4.618220e+00s
  Time in Function.fn.__call__: 4.519772e+00s (97.868%)
  Time in thunks: 4.468524e+00s (96.759%)
  Total compile time: 1.687160e+00s
    Number of Apply nodes: 26
    Theano Optimizer time: 1.493099e+00s
       Theano validate time: 4.654884e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 1.448960e-01s
       Import time 9.179330e-02s
       Node make_thunk time 1.380470e-01s
           Node Elemwise{Composite{exp((i0 * i1))}}(TensorConstant{(1, 1) of -2.0}, InplaceDimShuffle{x,0}.0) time 1.914692e-02s
           Node Elemwise{Composite{(Switch(Cast{int8}(GE(i0, i1)), (i2 - (i3 * i0)), i4) + i5)}}(sigma, TensorConstant{0}, TensorConstant{3.9120230674743652}, TensorConstant{50.0}, TensorConstant{-inf}, sigma_log__) time 1.753592e-02s
           Node Elemwise{add,no_inpl

In [4]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /Users/ckrapu/miniconda3/envs/pymc3-dev/lib/python3.7/site-packages/pymc3-3.5-py3.7.egg/pymc3/model.py:910
  Time in 1000 calls to Function.__call__: 1.147447e+01s
  Time in Function.fn.__call__: 1.134008e+01s (98.829%)
  Time in thunks: 1.043050e+01s (90.902%)
  Total compile time: 5.045708e+00s
    Number of Apply nodes: 49
    Theano Optimizer time: 2.870366e+00s
       Theano validate time: 1.429296e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.071112e+00s
       Import time 1.761718e-01s
       Node make_thunk time 2.066094e+00s
           Node Alloc(Elemwise{switch,no_inplace}.0, TensorConstant{401}, Shape_i{0}.0) time 1.797580e+00s
           Node Elemwise{Composite{Switch(i0, (i1 * (i2 - i3)), i4)}}(Elemwise{Composite{Cast{int8}(GT(i0, i1))}}.0, InplaceDimShuffle{x}.0, Subtensor{int64::}.0, Subtensor{:int64:}.0, TensorConstant{(1,) of 0}) time 1.598787e-02s
           Node Elemwise{Composite{(Switch(Cast{int8}(GE(i0, i